In [1]:
import os
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col, from_json, transform, regexp_extract, when, lit, floor, concat, count
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, ArrayType

In [2]:
os.getcwd()

'/home/msds2025/jvalera/bdcc2025/bdcc-lab-openlib/notebooks'

In [3]:
os.chdir('/home/msds2025/jvalera/bdcc2025/bdcc-lab-openlib/')

In [4]:
def human_readable_size(size_bytes):
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if size_bytes < 1024:
            return f"{size_bytes:.2f} {unit}"
        size_bytes /= 1024

In [5]:
file_path = "/mnt/data/public/openlibrary/20250426/ol_cdump_latest.txt.gz"
file_size_bytes = os.path.getsize(file_path)
print(f"ol_cdump_latest.txt.gz: {human_readable_size(file_size_bytes)}")

ol_cdump_latest.txt.gz: 41.89 GB


# Initialize Spark

In [6]:
# Initialize Spark session
spark = (SparkSession
         .builder
         .appName("Open Library Dataframes Creation")
         .master('local[*]')
         .getOrCreate()
        )

# Prepare Dataframes

In [7]:
pd.set_option("display.max_colwidth",None)
pd.set_option("display.max_columns",None)

In [8]:
file_path = "/mnt/data/public/openlibrary/20250426/ol_cdump_latest.txt.gz"

df = spark.read.text(file_path)

In [9]:
df.rdd.getNumPartitions()

1

In [10]:
df_split = (df
            .select(split(col("value"), "\t").alias("fields"))
            .select(
                    col("fields")[0].alias("record_type"),
                    col("fields")[1].alias("key"),
                    col("fields")[2].cast(IntegerType()).alias("revision"),
                    col("fields")[3].cast(TimestampType()).alias("timestamp"),
                    col("fields")[4].alias("json_data")
            )
           )

df_split.limit(10).toPandas()

,record_type,key,revision,timestamp,json_data
0,/type/author,/authors/OL5345273A,1,2008-10-05 09:14:51.676778,"{""name"": ""TOM.KIYIHN"", ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2008-10-05T09:14:51.676778""}, ""key"": ""/authors/OL5345273A"", ""type"": {""key"": ""/type/author""}, ""revision"": 1}"
1,/type/author,/authors/OL5345271A,1,2008-10-04 15:55:36.974594,"{""name"": ""Aksakov, S. T."", ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2008-10-04T15:55:36.974594""}, ""key"": ""/authors/OL5345271A"", ""type"": {""key"": ""/type/author""}, ""revision"": 1}"
2,/type/author,/authors/OL5345272A,1,2008-10-04 16:04:08.619520,"{""name"": ""Latham,Barbarahttp://webcat.camosun.bc.ca/cambooks/njpm/tableof.htm"", ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2008-10-04T16:04:08.61952""}, ""key"": ""/authors/OL5345272A"", ""type"": {""key"": ""/type/author""}, ""revision"": 1}"
3,/type/edition,/books/OL17806216M,1,2008-10-01 06:49:20.851055,"{""table_of_contents"": [{""type"": ""/type/text"", ""value"": ""Gender as a category of analysis in vernacular architect studies / Angel Kwolek-Folland""}, {""type"": ""/type/text"", ""value"": ""\""I'm a lumberjack and I'm ok\"" : the built environment and varied masculinities in the industrial age / Deryck W. Holdsworth""}, {""type"": ""/type/text"", ""value"": ""The Masonic lodge room, 1870-1930 : a sacred space of masculine spiritual hierarchy / William D. Moore""}, {""type"": ""/type/text"", ""value"": ""Grammar codes, and performance : linguistic and sociolinguistic models in the study of vernacular architecture / Michael Ann Williams and M. Jane Young""}, {""type"": ""/type/text"", ""value"": ""Building an urban identity : the clustered spires of Frederick, Maryland / Diane Shaw""}, {""type"": ""/type/text"", ""value"": ""\""A school house well arranged\"" : Baltimore public school buildings on the Lancasterian plan, 1829-1839 / Peter E. Kurtze""}, {""type"": ""/type/text"", ""value"": ""Letting in \""the world\"" : (re)interpretive tensions in the Quaker meeting house / Susan Garfinkel""}, {""type"": ""/type/text"", ""value"": ""Man\u0303ana, man\u0303ana : racial stereotypes and the Anglo rediscovery of the Southwest's vernacular architecture, 1890-1920 / Abigail A. Van Slyck""}, {""type"": ""/type/text"", ""value"": ""Association, residence, and shop : an appropriation of commercial blocks in North American Chinatowns / Christopher L. Yip""}, {""type"": ""/type/text"", ""value"": ""\""Snug li'l house with flue and oven\"" : nineteenth-century reforms in plantation slave housing / John Michael Vlach""}, {""type"": ""/type/text"", ""value"": ""Cheap and tasteful dwellings in popular architecture / Jan Jennings""}, {""type"": ""/type/text"", ""value"": ""Cheap, quick, and easy, part II : pressed metal ceilings, 1880-1930 / Pamela H. Simpson""}, {""type"": ""/type/text"", ""value"": ""The Eichler home : intention and experience in postwar suburbia / Annmarie Adams""}, {""type"": ""/type/text"", ""value"": ""Rural adaptations of suburban bungalows, Sussex County, Delaware / Susan Mulchahey Chase""}, {""type"": ""/type/text"", ""value"": ""Building in stone in southwestern Pennyslvania : patterns and process / Karen Koegler""}, {""type"": ""/type/text"", ""value"": ""Private dwellings, public ways, and the landscape of early rural capitalism in Virginia's Shenandoah Valley / Warren R. Hofstra""}, {""type"": ""/type/text"", ""value"": ""The architectural and social topography of early-nineteenth-century Portsmouth, New Hampshire / Bernard L. Herman""}, {""type"": ""/type/text"", ""value"": ""From roadside camps to garden homes : housing and community planning for California's migrant work force, 1935-1941 / Greg Hise.""}], ""series"": [""Perspectives in vernacular architecture -- 5."", ""Perspectives in vernacular architecture (Knoxville, Tenn.) -- 5.""], ""lc_classifications"": [""NA705 .G36 1995""], ""contributions"": [""Cromley, Elizabet

## Works Dataframe

In [11]:
df_works = df_split.filter(col("record_type") == "/type/work")
df_works.limit(10).toPandas()

,record_type,key,revision,timestamp,json_data
0,/type/work,/works/OL5967449W,3,2013-08-23 21:48:46.541299,"{""created"": {""type"": ""/type/datetime"", ""value"": ""2009-12-10T18:16:15.578495""}, ""subject_places"": [""Texas""], ""subjects"": [""Folk music"", ""History and criticism""], ""latest_revision"": 3, ""key"": ""/works/OL5967449W"", ""title"": ""Music in Texas--frontier to 1900"", ""authors"": [{""type"": {""key"": ""/type/author_role""}, ""author"": {""key"": ""/authors/OL1481792A""}}, {""type"": {""key"": ""/type/author_role""}, ""author"": {""key"": ""/authors/OL1461795A""}}], ""type"": {""key"": ""/type/work""}, ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2013-08-23T21:48:46.541299""}, ""revision"": 3}"
1,/type/work,/works/OL5918506W,3,2013-08-23 21:49:55.575503,"{""created"": {""type"": ""/type/datetime"", ""value"": ""2009-12-10T18:11:13.513027""}, ""subject_places"": [""Beaumont (Tex.)""], ""subjects"": [""History""], ""latest_revision"": 3, ""key"": ""/works/OL5918506W"", ""title"": ""Beaumont, a chronicle of promise"", ""authors"": [{""type"": {""key"": ""/type/author_role""}, ""author"": {""key"": ""/authors/OL1461795A""}}, {""type"": {""key"": ""/type/author_role""}, ""author"": {""key"": ""/authors/OL1481792A""}}], ""type"": {""key"": ""/type/work""}, ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2013-08-23T21:49:55.575503""}, ""revision"": 3}"
2,/type/work,/works/OL10007810W,3,2010-04-28 06:54:19.472104,"{""title"": ""La cuisine a la plancha (connaitre)"", ""created"": {""type"": ""/type/datetime"", ""value"": ""2009-12-11T01:58:28.508302""}, ""covers"": [3156622], ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2010-04-28T06:54:19.472104""}, ""latest_revision"": 3, ""key"": ""/works/OL10007810W"", ""authors"": [{""type"": ""/type/author_role"", ""author"": {""key"": ""/authors/OL3973957A""}}], ""type"": {""key"": ""/type/work""}, ""revision"": 3}"
3,/type/work,/works/OL9172590W,4,2013-08-23 22:18:31.678649,"{""title"": ""Cloud Dancer"", ""created"": {""type"": ""/type/datetime"", ""value"": ""2009-12-10T23:58:01.797117""}, ""covers"": [4967856], ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2013-08-23T22:18:31.678649""}, ""latest_revision"": 4, ""key"": ""/works/OL9172590W"", ""authors"": [{""type"": {""key"": ""/type/author_role""}, ""author"": {""key"": ""/authors/OL2674508A""}}], ""type"": {""key"": ""/type/work""}, ""revision"": 4}"
4,/type/work,/works/OL17802418W,1,2017-10-31 23:48:21.594355,"{""title"": ""Leer y redactar en la universidad : del caos de las ideas al texto estructurado"", ""created"": {""type"": ""/type/datetime"", ""value"": ""2017-10-31T23:48:21.594355""}, ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2017-10-31T23:48:21.594355""}, ""latest_revision"": 1, ""key"": ""/works/OL17802418W"", ""authors"": [{""type"": {""key"": ""/type/author_role""}}], ""type"": {""key"": ""/type/work""}, ""revision"": 1}"
5,/type/work,/works/OL17802421W,1,2017-11-01 03:39:02.464948,"{""title"": ""HELLBURNER"", ""created"": {""type"": ""/type/datetime"", ""value"": ""2017-11-01T03:39:02.464948""}, ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2017-11-01T03:39:02.464948""}, ""latest_revision"": 1, ""key"": ""/works/OL17802421W"", ""authors"": [{""type"": {""key"": ""/type/author_role""}, ""author"": {""key"": ""/authors/OL7046811A""}}], ""type"": {""key"": ""/type/work""}, ""revision"": 1}"
6,/type/work,/works/OL16806239W,1,2013-08-23 22:51:32.436808,"{""title"": ""An Alef-Bet Kabalah"", ""created"": {""type"": ""/type/datetime"", ""value"": ""2013-08-23T22:51:32.436808""}, ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2013-08-23T22:51:32.436808""}, ""latest_revision"": 1, ""key"": ""/works/OL16806239W"", ""authors"": [{""type"": {""key"": ""/type/author_role""}}], ""type"": {""key"": ""/type/work""}, ""revision"": 1}"
7,/type/work,/works/OL16806240W,1,2013-08-23 23:02:41.862836,"{""titl

In [12]:
works_json_schema = StructType([
    StructField("key", StringType(), True),
    StructField("title", StringType(), True),
    StructField("subtitle", StringType(), True),
    StructField("type", StructType([
        StructField("key", StringType(), True)
    ]), True),
    StructField("authors", ArrayType(StructType([
        StructField("type", StructType([
            StructField("key", StringType(), True)
        ]), True),
        StructField("author", StructType([
            StructField("key", StringType(), True)
        ]), True),
        StructField("role", StringType(), True),
        StructField("as", StringType(), True)
    ])), True),
    StructField("covers", ArrayType(IntegerType()), True),
    StructField("links", ArrayType(StructType([
        StructField("title", StringType(), True),
        StructField("url", StringType(), True)
    ])), True),
    StructField("id", IntegerType(), True),
    StructField("lc_classifications", ArrayType(StringType()), True),
    StructField("subjects", ArrayType(StringType()), True),
    StructField("first_publish_date", StringType(), True),
    StructField("description", StructType([
        StructField("type", StringType(), True),
        StructField("value", StringType(), True)
    ]), True),
    StructField("notes", StructType([
        StructField("type", StringType(), True),
        StructField("value", StringType(), True)
    ]), True),
    StructField("revision", IntegerType(), True),
    StructField("latest_revision", IntegerType(), True),
    StructField("created", StructType([
        StructField("type", StringType(), True),
        StructField("value", StringType(), True)
    ]), True),
    StructField("last_modified", StructType([
        StructField("type", StringType(), True),
        StructField("value", StringType(), True)
    ]), True)
])

In [13]:
df_works_parsed = df_works.withColumn(
    "json_parsed", from_json(col("json_data"), works_json_schema)
)

In [14]:
df_works_final = df_works_parsed.select(
    col("record_type"),
    col("key").alias("work_key"),
    col("json_parsed.title").alias("title"),
    col("json_parsed.subtitle").alias("subtitle"),
    transform(
        col("json_parsed.authors"),
        lambda x: x.author.key
    ).alias("author_keys"),
    col("json_parsed.subjects").alias("subjects"),
    col("json_parsed.first_publish_date").alias("first_publish_date")
)

In [15]:
df_works_final.limit(10).toPandas()

,record_type,work_key,title,subtitle,author_keys,subjects,first_publish_date
0,/type/work,/works/OL5967449W,Music in Texas--frontier to 1900,None,"[/authors/OL1481792A, /authors/OL1461795A]","[Folk music, History and criticism]",None
1,/type/work,/works/OL5918506W,"Beaumont, a chronicle of promise",None,"[/authors/OL1461795A, /authors/OL1481792A]",[History],None
2,/type/work,/works/OL10007810W,La cuisine a la plancha (connaitre),None,[/authors/OL3973957A],None,None
3,/type/work,/works/OL9172590W,Cloud Dancer,None,[/authors/OL2674508A],None,None
4,/type/work,/works/OL17802418W,Leer y redactar en la universidad : del caos de las ideas al texto estructurado,None,[None],None,None
5,/type/work,/works/OL17802421W,HELLBURNER,None,[/authors/OL7046811A],None,None
6,/type/work,/works/OL16806239W,An Alef-Bet Kabalah,None,[None],None,None
7,/type/work,/works/OL16806240W,Che Forever,None,[/authors/OL274904A],None,None
8,/type/work,/works/OL17802473W,Apartamento en la Costa Brava,None,[/authors/OL1540485A],None,None
9,/type/work,/works/OL17802492W,Artisti romi în arte contemporana,None,[None],None,None


In [16]:
output_dir = os.path.join(os.getcwd(), "partitioned_data")
!rm -rf output_dir
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [17]:
works_output_path = f"{output_dir}/works_partitioned"
df_works_final.write.parquet(works_output_path, mode="overwrite")

In [18]:
df_works_final = spark.read.parquet(works_output_path)

In [19]:
df_works_final.rdd.getNumPartitions()

31

## Authors Dataframe

In [20]:
df_authors = df_split.filter(col("record_type") == "/type/author")
df_authors.limit(10).toPandas()

,record_type,key,revision,timestamp,json_data
0,/type/author,/authors/OL5345273A,1,2008-10-05 09:14:51.676778,"{""name"": ""TOM.KIYIHN"", ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2008-10-05T09:14:51.676778""}, ""key"": ""/authors/OL5345273A"", ""type"": {""key"": ""/type/author""}, ""revision"": 1}"
1,/type/author,/authors/OL5345271A,1,2008-10-04 15:55:36.974594,"{""name"": ""Aksakov, S. T."", ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2008-10-04T15:55:36.974594""}, ""key"": ""/authors/OL5345271A"", ""type"": {""key"": ""/type/author""}, ""revision"": 1}"
2,/type/author,/authors/OL5345272A,1,2008-10-04 16:04:08.619520,"{""name"": ""Latham,Barbarahttp://webcat.camosun.bc.ca/cambooks/njpm/tableof.htm"", ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2008-10-04T16:04:08.61952""}, ""key"": ""/authors/OL5345272A"", ""type"": {""key"": ""/type/author""}, ""revision"": 1}"
3,/type/author,/authors/OL5345269A,1,2008-10-03 20:38:12.544508,"{""name"": ""Bi"", ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2008-10-03T20:38:12.544508""}, ""key"": ""/authors/OL5345269A"", ""type"": {""key"": ""/type/author""}, ""revision"": 1}"
4,/type/author,/authors/OL5345270A,1,2008-10-03 20:38:37.118889,"{""name"": ""Bianco Nicholas"", ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2008-10-03T20:38:37.118889""}, ""key"": ""/authors/OL5345270A"", ""type"": {""key"": ""/type/author""}, ""revision"": 1}"
5,/type/author,/authors/OL5345265A,1,2008-10-02 21:00:12.012264,"{""name"": ""Crisp, Tony"", ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2008-10-02T21:00:12.012264""}, ""key"": ""/authors/OL5345265A"", ""type"": {""key"": ""/type/author""}, ""revision"": 1}"
6,/type/author,/authors/OL5345268A,1,2008-10-03 13:50:01.827605,"{""name"": ""Jean-Pierre ROSSIE"", ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2008-10-03T13:50:01.827605""}, ""key"": ""/authors/OL5345268A"", ""type"": {""key"": ""/type/author""}, ""revision"": 1}"
7,/type/author,/authors/OL5345264A,1,2008-10-02 21:00:12.066032,"{""name"": ""Crisp, Tony"", ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2008-10-02T21:00:12.066032""}, ""key"": ""/authors/OL5345264A"", ""type"": {""key"": ""/type/author""}, ""revision"": 1}"
8,/type/author,/authors/OL5345266A,1,2008-10-02 22:32:21.621743,"{""name"": ""Si"", ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2008-10-02T22:32:21.621743""}, ""key"": ""/authors/OL5345266A"", ""type"": {""key"": ""/type/author""}, ""revision"": 1}"
9,/type/author,/authors/OL5345267A,1,2008-10-02 22:32:39.685528,"{""name"": ""Sikundar,Sylvia"", ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2008-10-02T22:32:39.685528""}, ""key"": ""/authors/OL5345267A"", ""type"": {""key"": ""/type/author""}, ""revision"": 1}"


In [21]:
authors_json_schema = StructType([
    StructField("key", StringType(), True),
    StructField("name", StringType(), True),
    StructField("location", StringType(), True),
    StructField("type", StructType([
        StructField("key", StringType(), True)
    ]), True),
    StructField("revision", IntegerType(), True),
    StructField("last_modified", StructType([
        StructField("type", StringType(), True),
        StructField("value", StringType(), True)
    ]), True)
])

In [22]:
df_authors_parsed = df_authors.withColumn(
    "json_parsed", from_json(col("json_data"), authors_json_schema)
)

In [23]:
df_authors_final = df_authors_parsed.select(
    col("record_type"),
    col("key").alias("author_key"),
    col("revision"),
    col("timestamp"),
    col("json_parsed.name").alias("name"),
    col("json_parsed.location").alias("location")
)

In [24]:
df_authors_final.limit(10).toPandas()

,record_type,author_key,revision,timestamp,name,location
0,/type/author,/authors/OL5345273A,1,2008-10-05 09:14:51.676778,TOM.KIYIHN,None
1,/type/author,/authors/OL5345271A,1,2008-10-04 15:55:36.974594,"Aksakov, S. T.",None
2,/type/author,/authors/OL5345272A,1,2008-10-04 16:04:08.619520,"Latham,Barbarahttp://webcat.camosun.bc.ca/cambooks/njpm/tableof.htm",None
3,/type/author,/authors/OL5345269A,1,2008-10-03 20:38:12.544508,Bi,None
4,/type/author,/authors/OL5345270A,1,2008-10-03 20:38:37.118889,Bianco Nicholas,None
5,/type/author,/authors/OL5345265A,1,2008-10-02 21:00:12.012264,"Crisp, Tony",None
6,/type/author,/authors/OL5345268A,1,2008-10-03 13:50:01.827605,Jean-Pierre ROSSIE,None
7,/type/author,/authors/OL5345264A,1,2008-10-02 21:00:12.066032,"Crisp, Tony",None
8,/type/author,/authors/OL5345266A,1,2008-10-02 22:32:21.621743,Si,None
9,/type/author,/authors/OL5345267A,1,2008-10-02 22:32:39.685528,"Sikundar,Sylvia",None


In [27]:
authors_output_path = f"{output_dir}/authors_partitioned"
df_authors_final.write.parquet(authors_output_path, mode="overwrite")

In [28]:
df_authors_final = spark.read.parquet(authors_output_path)

In [29]:
df_authors_final.rdd.getNumPartitions()

16

## Editions Dataframe

In [30]:
df_eds = df_split.filter(col("record_type") == "/type/edition")
df_eds.limit(10).toPandas()

,record_type,key,revision,timestamp,json_data
0,/type/edition,/books/OL17806216M,1,2008-10-01 06:49:20.851055,"{""table_of_contents"": [{""type"": ""/type/text"", ""value"": ""Gender as a category of analysis in vernacular architect studies / Angel Kwolek-Folland""}, {""type"": ""/type/text"", ""value"": ""\""I'm a lumberjack and I'm ok\"" : the built environment and varied masculinities in the industrial age / Deryck W. Holdsworth""}, {""type"": ""/type/text"", ""value"": ""The Masonic lodge room, 1870-1930 : a sacred space of masculine spiritual hierarchy / William D. Moore""}, {""type"": ""/type/text"", ""value"": ""Grammar codes, and performance : linguistic and sociolinguistic models in the study of vernacular architecture / Michael Ann Williams and M. Jane Young""}, {""type"": ""/type/text"", ""value"": ""Building an urban identity : the clustered spires of Frederick, Maryland / Diane Shaw""}, {""type"": ""/type/text"", ""value"": ""\""A school house well arranged\"" : Baltimore public school buildings on the Lancasterian plan, 1829-1839 / Peter E. Kurtze""}, {""type"": ""/type/text"", ""value"": ""Letting in \""the world\"" : (re)interpretive tensions in the Quaker meeting house / Susan Garfinkel""}, {""type"": ""/type/text"", ""value"": ""Man\u0303ana, man\u0303ana : racial stereotypes and the Anglo rediscovery of the Southwest's vernacular architecture, 1890-1920 / Abigail A. Van Slyck""}, {""type"": ""/type/text"", ""value"": ""Association, residence, and shop : an appropriation of commercial blocks in North American Chinatowns / Christopher L. Yip""}, {""type"": ""/type/text"", ""value"": ""\""Snug li'l house with flue and oven\"" : nineteenth-century reforms in plantation slave housing / John Michael Vlach""}, {""type"": ""/type/text"", ""value"": ""Cheap and tasteful dwellings in popular architecture / Jan Jennings""}, {""type"": ""/type/text"", ""value"": ""Cheap, quick, and easy, part II : pressed metal ceilings, 1880-1930 / Pamela H. Simpson""}, {""type"": ""/type/text"", ""value"": ""The Eichler home : intention and experience in postwar suburbia / Annmarie Adams""}, {""type"": ""/type/text"", ""value"": ""Rural adaptations of suburban bungalows, Sussex County, Delaware / Susan Mulchahey Chase""}, {""type"": ""/type/text"", ""value"": ""Building in stone in southwestern Pennyslvania : patterns and process / Karen Koegler""}, {""type"": ""/type/text"", ""value"": ""Private dwellings, public ways, and the landscape of early rural capitalism in Virginia's Shenandoah Valley / Warren R. Hofstra""}, {""type"": ""/type/text"", ""value"": ""The architectural and social topography of early-nineteenth-century Portsmouth, New Hampshire / Bernard L. Herman""}, {""type"": ""/type/text"", ""value"": ""From roadside camps to garden homes : housing and community planning for California's migrant work force, 1935-1941 / Greg Hise.""}], ""series"": [""Perspectives in vernacular architecture -- 5."", ""Perspectives in vernacular architecture (Knoxville, Tenn.) -- 5.""], ""lc_classifications"": [""NA705 .G36 1995""], ""contributions"": [""Cromley, Elizabeth C."", ""Hudgins, Carter L.""], ""edition_name"": ""1st ed."", ""title"": ""Gender, class, and shelter"", ""languages"": [{""key"": ""/languages/eng""}], ""subjects"": [""Vernacular architecture -- United States."", ""Architecture and society -- United States.""], ""publish_country"": ""tnu"", ""by_statement"": ""edited by Elizabeth Collins Cromley and Carter L. Hudgins."", ""type"": {""key"": ""/type/edition""}, ""revision"": 1, ""publishers"": [""University of Tennessee Press""], ""last_modified"": {""type"": ""/type/datetime"", ""value"": ""2008-10-01T06:49:20.851055""}, ""key"": ""/books/OL17806216M"", ""publish_places"": [""Knoxville""], ""pagination"": ""xiv, 272 p. :"", ""dewey_decimal_class"": [""720/.1/060973""], ""notes"": {""type"": ""/type/text"", ""value"": ""Includes bibliographical references (p. [259]-264) and index.""}, ""number_of_pages"": 272, ""lccn"": [""94

In [31]:
editions_json_schema = StructType([
    StructField("key", StringType(), True),
    StructField("title", StringType(), True),
    StructField("subtitle", StringType(), True),
    StructField("type", StructType([
        StructField("key", StringType(), True)
    ]), True),
    StructField("authors", ArrayType(StructType([
        StructField("key", StringType(), True)
    ])), True),
    StructField("works", ArrayType(StructType([
        StructField("key", StringType(), True)
    ])), True),
    StructField("identifiers", StructType([]), True),  # Generic object, skipped for now
    StructField("isbn_10", ArrayType(StringType()), True),
    StructField("isbn_13", ArrayType(StringType()), True),
    StructField("lccn", ArrayType(StringType()), True),
    StructField("ocaid", StringType(), True),
    StructField("oclc_numbers", ArrayType(StringType()), True),
    StructField("local_id", ArrayType(StringType()), True),
    StructField("covers", ArrayType(IntegerType()), True),
    StructField("links", ArrayType(StructType([
        StructField("title", StringType(), True),
        StructField("url", StringType(), True)
    ])), True),
    StructField("languages", ArrayType(StructType([
        StructField("key", StringType(), True)
    ])), True),
    StructField("translated_from", ArrayType(StructType([
        StructField("key", StringType(), True)
    ])), True),
    StructField("translation_of", StringType(), True),
    StructField("by_statement", StringType(), True),
    StructField("weight", StringType(), True),
    StructField("edition_name", StringType(), True),
    StructField("number_of_pages", IntegerType(), True),
    StructField("pagination", StringType(), True),
    StructField("physical_dimensions", StringType(), True),
    StructField("physical_format", StringType(), True),
    StructField("copyright_date", StringType(), True),
    StructField("publish_country", StringType(), True),
    StructField("publish_date", StringType(), True),
    StructField("publish_places", ArrayType(StringType()), True),
    StructField("publishers", ArrayType(StringType()), True),
    StructField("contributions", ArrayType(StringType()), True),
    StructField("dewey_decimal_class", ArrayType(StringType()), True),
    StructField("genres", ArrayType(StringType()), True),
    StructField("lc_classifications", ArrayType(StringType()), True),
    StructField("other_titles", ArrayType(StringType()), True),
    StructField("series", ArrayType(StringType()), True),
    StructField("source_records", ArrayType(StringType()), True),
    StructField("subjects", ArrayType(StringType()), True),
    StructField("work_titles", ArrayType(StringType()), True),
    StructField("table_of_contents", ArrayType(StringType()), True),
    StructField("description", StructType([
        StructField("type", StringType(), True),
        StructField("value", StringType(), True)
    ]), True),
    StructField("first_sentence", StructType([
        StructField("type", StringType(), True),
        StructField("value", StringType(), True)
    ]), True),
    StructField("notes", StructType([
        StructField("type", StringType(), True),
        StructField("value", StringType(), True)
    ]), True),
    StructField("revision", IntegerType(), True),
    StructField("latest_revision", IntegerType(), True),
    StructField("created", StructType([
        StructField("type", StringType(), True),
        StructField("value", StringType(), True)
    ]), True),
    StructField("last_modified", StructType([
        StructField("type", StringType(), True),
        StructField("value", StringType(), True)
    ]), True)
])

In [32]:
df_eds_parsed = df_eds.withColumn(
    "json_parsed", from_json(col("json_data"), editions_json_schema)
)

In [33]:
df_eds_final = df_eds_parsed.select(
    col("record_type"),
    col("key").alias("edition_key"),
    col("json_parsed.title").alias("title"),
    col("json_parsed.subtitle").alias("subtitle"),
    # Simplify authors to array of keys
    transform(
        col("json_parsed.authors"),
        lambda x: x.key
    ).alias("author_keys"),
    col("json_parsed.works").alias("work_keys"),
    # Simplify languages to array of keys
    transform(
        col("json_parsed.languages"),
        lambda x: x.key
    ).alias("language_keys"),
    col("json_parsed.publish_country").alias("publish_country"),
    col("json_parsed.publish_date").alias("publish_date"),
    col("json_parsed.publish_places").alias("publish_places"),
    col("json_parsed.genres").alias("genres"),
    col("json_parsed.subjects").alias("subjects"),
)

In [34]:
df_eds_final.limit(10).toPandas()

,record_type,edition_key,title,subtitle,author_keys,work_keys,language_keys,publish_country,publish_date,publish_places,genres,subjects
0,/type/edition,/books/OL17806216M,"Gender, class, and shelter",None,None,None,[/languages/eng],tnu,1995,[Knoxville],None,"[Vernacular architecture -- United States., Architecture and society -- United States.]"
1,/type/edition,/books/OL17841395M,Elizabeth Blackburn and the story of telomeres,None,[/authors/OL5345253A],None,None,None,None,"[MIT, 2007]",None,[History of Medicine]
2,/type/edition,/books/OL17841396M,Oregon Regional Primate Research Center facts.,None,[/authors/OL4684114A],None,[/languages/eng],oru,1962,"[Portland, Or]",None,"[Oregon Regional Primate Research Center., Primates., Research -- Oregon.]"
3,/type/edition,/books/OL17841387M,Pharmacologic management of heart failure and left ventricular systolic dysfunction,"effect in female, black, and diabetic patients, and cost-effectiveness",None,None,[/languages/eng],mdu,2003,"[Rockville, Md]",None,"[Heart failure -- Chemotherapy -- Evaluation., Heart failure -- Chemotherapy -- Cost effectiveness., Adrenergic beta blockers -- Therapeutic use -- Evaluation., Angiotensin converting enzyme -- Inhibitors -- Therapeutic use -- Evaluation., Diabetes -- Complications -- Chemotherapy -- Evaluation., Evidence-based medicine., Heart Failure -- drug therapy., Ventricular Dysfunction, Left -- drug therapy., Evidence-Based Medicine -- methods.]"
4,/type/edition,/books/OL17841388M,Disease-germs,None,[/authors/OL2456514A],None,[/languages/eng],cau,1879,[S.F. [i.e. San Francisco]],None,"[Germ theory of disease., Bacteria., Bacteria.]"
5,/type/edition,/books/OL17841389M,Current diagnosis and treatment: emergency medicine,None,[/authors/OL5345248A],None,None,None,None,"[McGraw Hill, 2008]",None,[Emergency Medicine]
6,/type/edition,/books/OL17841390M,Midwifery & Childbirth in America,None,[/authors/OL5345249A],None,None,None,None,None,None,None
7,/type/edition,/books/OL17841391M,Fundamentals of musculoskeletal ultrasound,None,[/authors/OL5345250A],None,None,None,None,"[Saunders, 2007]",None,[Radiology]
8,/type/edition,/books/OL17841366M,Tami's test record,None,[/authors/OL5345237A],None,None,None,None,[2007],None,None
9,/type/edition,/books/OL17841392M,"History of the Department of Surgery, 1867-2007",None,[/authors/OL5345251A],None,[/languages/eng],oru,2007,"[Portland, Or]",None,"[Oregon Health & Science University. -- Dept. of Surgery., Oregon Health Sciences University. -- Dept. of Surgery., University of Oregon Health Sciences Center. -- Dept. of Surgery., University of Oregon. -- Medical School. -- Dept. of Surgery., Surgery -- history -- Oregon., Schools, Medical -- history -- Oregon.]"


In [35]:
eds_output_path = f"{output_dir}/eds_partitioned"
df_eds_final.write.parquet(eds_output_path, mode="overwrite")

In [36]:
df_eds_final = spark.read.parquet(eds_output_path)

In [37]:
df_eds_final.rdd.getNumPartitions()

103

In [38]:
spark.stop()

In [39]:
total_size = 0.0
for i, file in enumerate(os.listdir('/home/msds2025/jvalera/bdcc2025/bdcc-lab-openlib/partitioned_data/eds_partitioned/')):
    file_path = '/home/msds2025/jvalera/bdcc2025/bdcc-lab-openlib/partitioned_data/eds_partitioned/' + file
    file_size_bytes = os.path.getsize(file_path)
    total_size += file_size_bytes
    print(f"{file}: {human_readable_size(file_size_bytes)}")
print(f"Total Size: {human_readable_size(total_size)}")

_SUCCESS: 0.00 B
part-00000-ac7e19f9-82eb-49d5-a0eb-c87bf900c2a5-c000.snappy.parquet: 12.86 GB
.part-00000-ac7e19f9-82eb-49d5-a0eb-c87bf900c2a5-c000.snappy.parquet.crc: 102.89 MB
._SUCCESS.crc: 8.00 B
Total Size: 12.96 GB


In [40]:
total_size = 0.0
for i, file in enumerate(os.listdir('/home/msds2025/jvalera/bdcc2025/bdcc-lab-openlib/partitioned_data/authors_partitioned/')):
    file_path = '/home/msds2025/jvalera/bdcc2025/bdcc-lab-openlib/partitioned_data/authors_partitioned/' + file
    file_size_bytes = os.path.getsize(file_path)
    total_size += file_size_bytes
    print(f"{file}: {human_readable_size(file_size_bytes)}")
print(f"Total Size: {human_readable_size(total_size)}")

_SUCCESS: 0.00 B
part-00000-1d12f884-9f4d-48b9-884d-37d03c766b68-c000.snappy.parquet: 446.62 MB
.part-00000-1d12f884-9f4d-48b9-884d-37d03c766b68-c000.snappy.parquet.crc: 3.49 MB
._SUCCESS.crc: 8.00 B
Total Size: 450.11 MB


In [41]:
total_size = 0.0
for i, file in enumerate(os.listdir('/home/msds2025/jvalera/bdcc2025/bdcc-lab-openlib/partitioned_data/works_partitioned/')):
    file_path = '/home/msds2025/jvalera/bdcc2025/bdcc-lab-openlib/partitioned_data/works_partitioned/' + file
    file_size_bytes = os.path.getsize(file_path)
    total_size += file_size_bytes
    print(f"{file}: {human_readable_size(file_size_bytes)}")
print(f"Total Size: {human_readable_size(total_size)}")

_SUCCESS: 0.00 B
part-00000-7a7024c9-2d18-4186-aab9-f9c08a405214-c000.snappy.parquet: 3.84 GB
.part-00000-7a7024c9-2d18-4186-aab9-f9c08a405214-c000.snappy.parquet.crc: 30.71 MB
._SUCCESS.crc: 8.00 B
Total Size: 3.87 GB
